## Quantizing 🤗 Transformers models with the GPTQ method can be done in a only few lines.

📌 Note that for large model like 175B param, at least 4 GPU-hours will be needed if one uses a large dataset (e.g. `"c4"``).

📌 Of course many GPTQ models are already available on the Hugging Face Hub, which bypasses the need to quantize a model yourself in most use cases. Nevertheless, you can also quantize a model using your own dataset appropriate for the particular domain you are working on.

In [ ]:
# Quantizing 🤗 Transformers models with the GPTQ method

from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig

model_id = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = GPTQConfig(bits=4, dataset = "c4", tokenizer=tokenizer)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto",
                                    quantization_config=quantization_config)

model.push_to_hub("username/Llama-2-7b-gptq-4bit")
tokenizer.push_to_hub("username/Llama-2-7b-gptq-4bit")

`bits` param: Its the number of bits to quantize to, supported numbers are (2, 3, 4, 8).

`dataset`: The dataset used for calibration. I would leave “c4“ which seems to yield reasonable results. Other datasets are supported according to the documentation.

`tokenizer`: The tokenizer of Llama 2 7B that will be applied to c4.

- `desc_act` : Whether to quantize columns in order of decreasing activation size. Setting it to False can significantly speed up inference but the perplexity may become slightly worse. If inference speed is not your concern, you should set `desc_act` to True.

- `use_exllama` - bool, optional — Whether to use exllama backend. Defaults to True if unset. Only works with bits = 4. For 4-bit model, you can use the exllama kernels in order to a faster inference speed.

You need to have the entire model on gpus if you want to `use_exllama` kernels. So if you plan to use the model on a configuration with a small VRAM that will split the model to multiple devices with `device_map`, you should set `use_exllama` to False.